In [0]:
# Process the CDC data using the SCD4 pattern
# SCD4 is similar to the SCD2 but we keep the active records in seperate table.
from pyspark.sql.functions import *
from pyspark.sql import Window

class SilverLayer_SCD4:
    def __init__(self):
        pass

    def read_raw_data(self):
        raw_df = (
            spark
            .readStream
            .table("DEV.scd_raw_layer.users_profile_raw")
        )
        return raw_df
    
    def process_raw_df(self,raw_df):
        processed_df = (
            raw_df
            .selectExpr("user_id","update_type","timestamp","dob","sex","gender","first_name","last_name","address.*")
            .withColumnRenamed("timestamp","event_timestamp")
            .withColumn("event_timestamp",col("event_timestamp").cast("timestamp"))
            .withColumn("dob",to_date(col("dob"),"mm/dd/yyyy"))
            .withColumn("start_time",col("event_timestamp"))
            .withColumn("end_time",lit(None))
        )
        return processed_df
    
    def upsert(self,df,batch_id):

        # Implement SCD Type1 in the actv table

        # pickup the latest records within the microbatch
        window_spec = Window.partitionBy("user_id").orderBy(col("event_timestamp").desc())
        df_latest = df.withColumn("row_num",row_number().over(window_spec))
        df_latest = df_latest.filter(col("row_num") == 1).drop("row_num","start_time","end_time")

        df_latest.createOrReplaceTempView("users_profile_scd1_4_stage")

        scd1_merge_statement = """
        MERGE into DEV.scd_silver_layer.users_profile_scd4_actv t
        using users_profile_scd1_4_stage s
        on s.user_id = t.user_id
        when matched and s.update_type = 'update' then update set *
        when matched and s.update_type = 'delete' then delete
        when not matched and s.update_type = 'new' then insert *
        when not matched then insert *
        """

        df_latest._jdf.sparkSession().sql(scd1_merge_statement)

        # Implement SCD Type2 in the hist table
        window_spec_scd2 = Window.partitionBy("user_id").orderBy(col("event_timestamp"))
        df_rnk = df.withColumn("row_num",row_number().over(window_spec_scd2))

        # Update the existing records
        scd_type2_update_statement = """
        MERGE into DEV.scd_silver_layer.users_profile_scd4_hist t
        using users_profile_update_scd2_4_stage s
        on s.user_id = t.user_id
        and s.start_time > t.start_time
        and t.end_time is null
        and s.update_type in ('update','delete')
        when matched then update set end_time = s.start_time
        """

        scd_type2_insert_statement = """
        MERGE into DEV.scd_silver_layer.users_profile_scd4_hist t
        using users_profile_insert_scd2_4__stage s
        on s.user_id = t.user_id
        and t.end_time is null
        when not matched 
        then insert (
         user_id,
         event_timestamp,
         dob,
         sex,
         gender ,
         first_name ,
         last_name ,
         street_address, 
         city, 
         state, 
         zip,
         start_time,
         end_time
        ) 
        values(
            s.user_id,
            s.event_timestamp,
            s.dob,
            s.sex,
            s.gender,
            s.first_name,
            s.last_name,
            s.street_address, 
            s.city, 
            s.state, 
            s.zip,
            s.start_time,
            s.end_time
        )
        """

        mx_rnk = df_rnk.select(max("row_num").alias("max_rnk")).collect()[0][0]
        for i in range(1,mx_rnk+1):
            t = df_rnk.filter(col("row_num") == i).drop("row_num")
            t.createOrReplaceTempView("users_profile_update_scd2_4_stage")
            t._jdf.sparkSession().sql(scd_type2_update_statement)

            t1=t.filter(col("update_type") != 'delete')
            t1.createOrReplaceTempView("users_profile_insert_scd2_4__stage")
            t1._jdf.sparkSession().sql(scd_type2_insert_statement)

    def write_processed_df(self,processed_df):
        streaming_query = (processed_df
                            .writeStream
                            .queryName("user_profile_scd_type4")
                            .option("checkpointLocation","/Volumes/dev/scd_silver_layer/users_profile_scd4")
                            .trigger(processingTime="1 seconds")
                            .outputMode("update")
                            .foreachBatch(self.upsert)
                            .start()
                          )
        
        return streaming_query
    
    def start_streaming_query(self):
        raw_df = self.read_raw_data()
        processed_df = self.process_raw_df(raw_df)
        streaming_query = self.write_processed_df(processed_df)
        return streaming_query

In [0]:
scd_type4 = SilverLayer_SCD4()
streaming_query_scd4 = scd_type4.start_streaming_query()

In [0]:
streaming_query_scd4.stop()

In [0]:
%sql
select * from DEV.scd_silver_layer.users_profile_scd4_actv;

user_id,event_timestamp,dob,sex,gender,first_name,last_name,street_address,city,state,zip
12140,2023-03-10T12:33:45Z,1999-01-02,M,M,Robert,Castillo,68994 Steven Vista,Pearblossom,CA,93553
12227,2023-03-10T12:40:28Z,1949-01-11,F,F,Courtney,Sheppard,47754 Angela Plaza Apt. 135,Los Angeles,CA,90010
14232,2023-03-11T12:51:27Z,1979-01-04,M,M,Edward,Simpson,92012 Bradley Shoals,Long Beach,CA,90815
14508,2023-03-11T12:50:25Z,1936-01-28,M,M,Justin,Eaton,04952 Lori Plain,Sierra Madre,CA,91024
14633,2023-03-11T12:47:10Z,1997-01-04,F,F,Hannah,Fuller,81346 Obrien Streets,Gardena,CA,90249


In [0]:
%sql
select * from DEV.scd_silver_layer.users_profile_scd4_hist;

user_id,event_timestamp,dob,sex,gender,first_name,last_name,street_address,city,state,zip,start_time,end_time
12227,2023-03-10T12:40:28Z,1949-01-11,F,F,Courtney,Sheppard,47754 Angela Plaza Apt. 135,Los Angeles,CA,90010,2023-03-10T12:40:28Z,null
14232,2023-03-11T12:51:27Z,1979-01-04,M,M,Edward,Simpson,92012 Bradley Shoals,Long Beach,CA,90815,2023-03-11T12:51:27Z,null
14508,2023-03-11T12:50:25Z,1936-01-28,M,M,Justin,Eaton,04952 Lori Plain,Sierra Madre,CA,91024,2023-03-11T12:50:25Z,null
14633,2023-03-11T12:47:10Z,1997-01-04,F,F,Hannah,Fuller,81346 Obrien Streets,Gardena,CA,90249,2023-03-11T12:47:10Z,null
12140,2023-03-10T12:32:09Z,1999-01-02,M,M,Robert,Clark,68994 Steven Vista,Pearblossom,CA,93553,2023-03-10T12:32:09Z,2023-03-10T12:33:45Z
12140,2023-03-10T12:33:45Z,1999-01-02,M,M,Robert,Castillo,68994 Steven Vista,Pearblossom,CA,93553,2023-03-10T12:33:45Z,null
